# Definitions of regions for contribution calculations


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import netCDF4 as nc
import warnings
warnings.filterwarnings('ignore')
import pickle

%matplotlib inline

#### Parameters:

In [ ]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799
isize = imax-imin
jsize = jmax-jmin

#### Load files:

In [ ]:
# ANHA12 grid:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask      = np.array(mesh.variables['tmask'][0,:,:,:])
Z_masked   = np.ma.masked_where((tmask > 0.1), tmask) 

#### Calculations:

In [ ]:
# Find indices to separate out Canada Basin:

x_ind = np.arange(1621, 2100, 1)
y_ind = (-7/8)*x_ind + 1517 + 700

CB_indx = []
CB_indy = []

for index in range(0,len(x_ind)):
    CB_x = np.arange(x_ind[index],2179,1)
    CB_y = np.ones(CB_x.shape)*y_ind[index]
    
    CB_indx = np.append(CB_x, CB_indx)
    CB_indy = np.append(CB_y, CB_indy)

In [ ]:
# Separate Canada Basin and the CAA:
mask_ini_CB  = np.zeros((isize,jsize))
mask_ini_CAA = np.ones((isize,jsize))

for i, j in zip(CB_indx, CB_indy):
    mask_ini_CB[int(i-imin),int(j-jmin)]  = 1
    mask_ini_CAA[int(i-imin),int(j-jmin)] = 0
    
mask_ini_CB[150:-1 ,-8:-1] = 1
mask_ini_CAA[150:-1,-8:-1] = 0

In [ ]:
mask_ini_wCAA = np.zeros((isize,jsize))
mask_ini_eCAA = np.zeros((isize,jsize))

# Separate the western and eastern CAA:
mask_ini_wCAA = np.where(mesh_lon[imin:imax,jmin:jmax] < -100, 1, 0)
mask_ini_eCAA = np.where(mesh_lon[imin:imax,jmin:jmax] > -100, 1, 0)

for i, j in zip(CB_indx, CB_indy):
    mask_ini_wCAA[int(i-imin),int(j-jmin)] = 0
    mask_ini_eCAA[int(i-imin),int(j-jmin)] = 0
    
mask_ini_wCAA[150:-1,-8:-1] = 0
mask_ini_eCAA[150:-1,-8:-1] = 0

##### Figures:

In [ ]:
fig, ax1, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/mn-reference.pickle','rb'))

# Sub-domain map: ---------------------------------------------------------------------------
x_sub, y_sub  = proj1(mesh_lon[imin:imax,jmin:jmax], mesh_lat[imin:imax,jmin:jmax]) 

# Draw domains:
proj1.plot(np.ma.masked_where(mask_ini_CB==0  , x_sub), np.ma.masked_where(mask_ini_CB==0  , y_sub),\
           '-', color='#304452', linewidth=0.4, zorder=1)
proj1.plot(np.ma.masked_where(mask_ini_wCAA==0, x_sub), np.ma.masked_where(mask_ini_wCAA==0, y_sub),\
           '-', color='#6c99b8', linewidth=0.4, zorder=1)
proj1.plot(np.ma.masked_where(mask_ini_eCAA==0, x_sub), np.ma.masked_where(mask_ini_eCAA==0, y_sub), \
           '-', color='#c9ddeb', linewidth=0.4, zorder=1)

x_full, y_full = proj1(mesh_lon, mesh_lat) 
proj1.plot(x_full[1935,450:530], y_full[1935,450:530], '--k', dashes=(1.0,0.75), linewidth=2.0, \
           label='Parry Channel boundaries')
proj1.plot(x_full[1585,485:538], y_full[1585,485:538], '--k', dashes=(1.0,0.75), linewidth=2.0)

# Legend entries
proj1.plot(0,0, '-', c='#304452', lw=5, label='Canada Basin');
proj1.plot(0,0, '-', c='#6c99b8', lw=5, label='Western Canadian Arctic Archipelago');
proj1.plot(0,0, '-', c='#c9ddeb', lw=5, label='Eastern Canadian Arctic Archipelago');

leg = ax1.legend(loc=(0.48,0.77), fontsize=5.8, framealpha=0.9, frameon=True,fancybox=False)
leg.get_frame().set_linewidth(0.0)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S14-region-definitions.jpg', bbox_inches='tight', dpi=300)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S14-region-definitions.svg', bbox_inches='tight', dpi=300, \
            format='svg')